In [1]:
import numpy as np
import pandas as pd
import sklearn.datasets
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import optuna
import optuna.integration.lightgbm as lgb
import math
from optuna.trial import Trial

In [4]:
np.random.seed(123)

# import data
data_transformed = pd.read_csv('data_transformed.csv')

# avoid this ugly slicing by using a two-dim dataset
X = data_transformed.iloc[:, :-1]
y = data_transformed.iloc[:, :]['Class']


# using 75% of the data for training and 30% for testing (with
# stratification for imbalanced class)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, stratify=y, random_state=123)

# Standardize features by removing the mean and scaling to unit variance
sc = StandardScaler()
scaled_X_train = sc.fit_transform(X_train)
scaled_X_test = sc.transform(X_test)

In [5]:
class LGBM():
  
  def __init__(self,scaled_X_train, scaled_X_test, y_train, y_test):
    dtrain = lgb.Dataset(scaled_X_train, label= y_train)
    dtest = lgb.Dataset(scaled_X_test, label=y_test)
    
  def objective(self, trial = Trial):
    dtrain = lgb.Dataset(scaled_X_train, label= y_train)
    dtest = lgb.Dataset(scaled_X_test, label=y_test)

    params = {
        "objective": "binary",
        "metric": "auc",
        "verbosity": 0,
        "boosting_type": "gbdt",
    }
    gbm = lgb.train(
        params, dtrain, verbose_eval=True, valid_sets=[dtest]
    )

    preds = gbm.predict(scaled_X_test)
    y_pred = np.array(list(map(lambda x: int(x), preds>0.5)))
    f1_sc = sklearn.metrics.f1_score(y_test, y_pred)
    loss = np.subtract(1,f1_sc)
    return loss

  def optuna_method(self):
    study = optuna.create_study(direction="minimize")
    study.optimize(self.objective, n_trials=2000)
    self.params = study.best_params
    return study.best_trial



In [6]:
x = LGBM(scaled_X_train, scaled_X_test, y_train, y_test)

In [ ]:
x.optuna_method()

[I 2021-04-30 14:46:30,097] A new study created in memory with name: no-name-34c9cdd2-b634-4644-82a3-4b06c2cdcf73
[I 2021-04-30 14:46:30,106] A new study created in memory with name: no-name-d264c04c-ab16-409e-bfbf-b0a91c93b132
feature_fraction, val_score: -inf:   0%|                                                         | 0/7 [00:00<?, ?it/s]

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007858 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1]	valid_0's auc: 0.983278
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2]	valid_0's auc: 0.983278
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3]	valid_0's auc: 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4]	valid_0's auc: 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[5]	valid_0's auc: 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[6]	valid_0's auc: 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[7]	valid_0's auc: 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[8]	valid_0's auc: 1
[LightGBM] [Warning] No further splits with positive 

[166]	valid_0's auc: 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[167]	valid_0's auc: 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[168]	valid_0's auc: 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[169]	valid_0's auc: 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[170]	valid_0's auc: 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[171]	valid_0's auc: 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[172]	valid_0's auc: 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[173]	valid_0's auc: 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[174]	valid_0's auc: 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[175]	valid_0's auc: 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[176]	valid_0's auc:

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[333]	valid_0's auc: 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[334]	valid_0's auc: 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[335]	valid_0's auc: 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[336]	valid_0's auc: 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[337]	valid_0's auc: 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[338]	valid_0's auc: 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[339]	valid_0's auc: 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[340]	valid_0's auc: 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[341]	valid_0's auc: 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[342]	valid_0's auc: 1
[LightGBM] [Warning]

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[499]	valid_0's auc: 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[500]	valid_0's auc: 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[501]	valid_0's auc: 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[502]	valid_0's auc: 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[503]	valid_0's auc: 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[504]	valid_0's auc: 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[505]	valid_0's auc: 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[506]	valid_0's auc: 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[507]	valid_0's auc: 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[508]	valid_0's auc: 1
[LightGBM] [Warning]

...

feature_fraction, val_score: 1.000000:  29%|############8                                | 2/7 [00:46<01:52, 22.55s/it]

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001207 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1]	valid_0's auc: 0.983278
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2]	valid_0's auc: 0.983278
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3]	valid_0's auc: 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4]	valid_0's auc: 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[5]	valid_0's auc: 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[6]	valid_0's auc: 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[7]	valid_0's auc: 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[8]	valid_0's auc: 1
[LightGBM] [Warning] No further splits with positive 

[166]	valid_0's auc: 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[167]	valid_0's auc: 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[168]	valid_0's auc: 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[169]	valid_0's auc: 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[170]	valid_0's auc: 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[171]	valid_0's auc: 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[172]	valid_0's auc: 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[173]	valid_0's auc: 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[174]	valid_0's auc: 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[175]	valid_0's auc: 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[176]	valid_0's auc:

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[333]	valid_0's auc: 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[334]	valid_0's auc: 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[335]	valid_0's auc: 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[336]	valid_0's auc: 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[337]	valid_0's auc: 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[338]	valid_0's auc: 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[339]	valid_0's auc: 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[340]	valid_0's auc: 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[341]	valid_0's auc: 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[342]	valid_0's auc: 1
[LightGBM] [Warning]

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[499]	valid_0's auc: 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[500]	valid_0's auc: 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[501]	valid_0's auc: 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[502]	valid_0's auc: 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[503]	valid_0's auc: 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[504]	valid_0's auc: 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[505]	valid_0's auc: 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[506]	valid_0's auc: 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[507]	valid_0's auc: 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[508]	valid_0's auc: 1
[LightGBM] [Warning]